# TP Sytèmes de recommandation


## 1 Présentation du modèle

In [1]:
import numpy as np
from numpy.linalg import norm
from scipy import sparse
from scipy.sparse.linalg import svds
from scipy.optimize import check_grad
from scipy.optimize import brent
from timeit import timeit

### Question 1.1

#### Que fait l'option `minidata` de la fonction `load_movielens` de `movielensutils.py` ?


In [2]:
def load_movielens(filename, minidata=False):
    """
    Cette fonction lit le fichier filename de la base de donnees
    Movielens, par exemple 
    filename = '~/datasets/ml-100k/u.data'
    Elle retourne 
    R : une matrice utilisateur-item contenant les scores
    mask : une matrice valant 1 si il y a un score et 0 sinon
    """

    data = np.loadtxt(filename, dtype=int)

    R = sparse.coo_matrix((data[:, 2], (data[:, 0]-1, data[:, 1]-1)),
                          dtype=float)
    R = R.toarray()  # not optimized for big data

    # code la fonction 1_K
    mask = sparse.coo_matrix((np.ones(data[:, 2].shape),
                              (data[:, 0]-1, data[:, 1]-1)), dtype=bool)
    mask = mask.toarray()  # not optimized for big data

    if minidata is True:
        R = R[0:100, 0:200].copy()
        mask = mask[0:100, 0:200].copy()

    return R, mask

L'option `minidata` réduit les données de travail aux notes des 100 premiers utilisateurs pour les 200 premiers films.

### Question 1.2

#### Combien y a-t-il d'utilisateurs, de films référencés dans la base de données ? Quel est le nombre total de notes ?

Il y a :
* 943 utilisateurs : nombre de lignes de R
* 1682 films : nombre de colonnes de R
* 100 000 notes : donné par `print(np.sum(m_mask))`

### Question 1.3

#### La fonction objectif est-elle convexe ?

Soit $f$ la fonction objectif : 
$
f: (P, Q) \mapsto \frac{1}{2}\|\mathbb{1}_K \circ (R - QP)\|_F^2 + \frac{\rho}{2}\|P\|_F^2 + \frac{\rho}{2}\|Q\|_F^2
$.
On remarque que $f$ est une forme quadratique.

Fixons $\rho = 0$ et $|U| = |C| = |I| = 1$. $f$ s'écrit alors : $f(p, q) = \frac{1}{2}(r - qp)^2$.

Soit $H(f)$ la matrice hessienne de f :
$
H(f) =
\begin{bmatrix}
q^2 & 2qp -r \\
2qp -r & p^2
\end{bmatrix}
$
, donc : $det(H(f)) = p^2q^2 - (2pq - r)^2 = (r - pq)(3pq - r)$ .

Pour $p = q = 0$ et $r = 1$, $det(H(f)) = -1$, la hessienne n'est donc pas définie positive, $f$ n'est donc pas convexe.

#### Quel est son gradient ?

$f(P, Q) = \frac{1}{2}\|\mathbb{1}_K \circ (R - QP)\|_F^2 + \frac{\rho}{2}\|P\|_F^2 + \frac{\rho}{2}\|Q\|_F^2 = f_1(P,Q) + f_2(P) + f_3(Q)$

$\nabla{f}_Q$ :

$
f_1(P+H, Q)-f_1(P, Q) = \frac{1}{2}(\|\mathbb{1}_K \circ (R-Q(P+H))\|_F^2 - \|\mathbb{1}_K \circ (R - QP)\|_F^2)
$  
$
= \frac{1}{2}(\|\mathbb{1}_K \circ (R-QP) - \mathbb{1}_K \circ QH)\|_F^2 - \|\mathbb{1}_K \circ (R - QP)\|_F^2)
$  
$
= \frac{1}{2}(\|\mathbb{1}_K \circ (R-QP)\|_F^2 + \|\mathbb{1}_K \circ QH)\|_F^2 
-2<\mathbb{1}_K \circ (R-QP),\mathbb{1}_K \circ QH>
- \|\mathbb{1}_K \circ (R - QP)\|_F^2)
$  
$
= - <Q^T(\mathbb{1}_K \circ (R-QP)), H> + o(H)
$

Donc $\nabla{f}_Q(P, Q) = - Q^T(\mathbb{1}_K \circ (R-QP)) + \rho P$.

$\nabla{f}_P$ :

$
f_1(P, Q+H)-f_1(P, Q) = \frac{1}{2}(\|\mathbb{1}_K \circ (R-(Q+H)P))\|_F^2 - \|\mathbb{1}_K \circ (R - QP)\|_F^2)
$  
$
= \frac{1}{2}(\|\mathbb{1}_K \circ (R-QP) - \mathbb{1}_K \circ HP)\|_F^2 - \|\mathbb{1}_K \circ (R - QP)\|_F^2)
$  
$
= \frac{1}{2}(\|\mathbb{1}_K \circ (R-QP)\|_F^2 + \|\mathbb{1}_K \circ HP)\|_F^2 
-2<\mathbb{1}_K \circ (R-QP),\mathbb{1}_K \circ HP>
- \|\mathbb{1}_K \circ (R - QP)\|_F^2)
$  
$
= - <(\mathbb{1}_K \circ (R-QP))P^T, H> + o(H)
$

Donc $\nabla{f}_P(P, Q) = - (\mathbb{1}_K \circ (R-QP))P^T + \rho P$.

Donc :

$$
\nabla{f}(P,Q) =
\begin{bmatrix}
\nabla{f_Q} \\
\nabla{f_P}
\end{bmatrix}
= \begin{bmatrix}
-Q^T(\mathbb{1}_K \circ (R - QP)) + \rho P \\
(\mathbb{1}_K \circ (R-QP))(-P^T) + \rho Q
\end{bmatrix}.
$$

#### Est-il lipschitzien ? Donner la constance de Lipschitz le cas échéant.

Si le gradient était Lipschitzien la hessienne serait bornée, or ici la hessienne est un polynôme d'ordre 2, $f$ n'est donc pas de gradient Lipschitzien.

## 2 Trouver $P$ quand $Q_0$ est fixé
$$
g: P \mapsto \frac{1}{2}\|\mathbb{1}_K \circ (R - Q^0P)\|_F^2 + \frac{\rho}{2}\|P\|_F^2 + \frac{\rho}{2}\|Q^0\|_F^2
$$

### Question 2.1

#### La fonction objectif $g$ est-elle convexe ?

La hessienne $\nabla^2{g}(P) = (Q^0)^TQ^0 + \rho I$ est définie positive pour $\rho$ convenablement choisi, g est donc convexe.

### Quel est son gradient ?

$$
\nabla{g}(P) = -(Q^0)^T(\mathbb{1}_K \circ (R-Q^0P)) + \rho P
$$

### Question 2.2

In [3]:
def objective(P, Q0, R, mask, rho):
    """
    La fonction objectif du probleme simplifie.
    Prend en entree 
    P : la variable matricielle de taille C x I
    Q0 : une matrice de taille U x C
    R : une matrice de taille U x I
    mask : une matrice 0-1 de taille U x I
    rho : un reel positif ou nul

    Sorties :
    val : la valeur de la fonction
    grad_P : le gradient par rapport a P
    """

    tmp = (R - Q0.dot(P)) * mask

    val = np.sum(tmp ** 2)/2. + rho/2. * (np.sum(Q0 ** 2) + np.sum(P ** 2))

    grad_P = -Q0.transpose().dot(tmp) + rho*P

    return val, grad_P

Vérification de l'implémentation :

In [4]:
R, mask = load_movielens('ml-100k/u.data')
Q0, s, P0 = svds(R, k=4)
rho = 0.3
check_grad(
    lambda p: objective(np.reshape(p, np.shape(P0)), Q0, R, mask, rho)[0],
    lambda p: np.ravel(objective(np.reshape(p, np.shape(P0)), Q0, R, mask, rho)[1]),
    np.ravel(P0))

1.1221547542271297

L'erreur étant négligeable, l'implémentation est satisfaisante.

### Question 2.3

In [5]:
def gradient(g, P0, gamma, epsilon):
    """
    Minimise g par la méthode du gradient a pas constant.
    Prend en entree
    g : fonction a minimiser, Pk -> g(Pk), grad_g(Pk)
    P0 : point de depart
    gamma : pas constant
    epsilon : critere d'arret, norme de Frobenius du gradient de g en Pk
              inferieur ou egal a epsilon
    
    Sorties :
    Pk : minimiseur
    """
    
    grad_G = g(P0)[1]
    Pk = P0
    while np.sum(grad_G**2) > epsilon:
        Pk = Pk - gamma*grad_G
        grad_G = g(Pk)[1]
    
    return Pk

### Question 2.4

$\epsilon = 1$

In [6]:
epsilon = 1
L = rho + np.sum(Q0.transpose().dot(Q0)**2)
gradient(lambda P: objective(P, Q0, R, mask, rho), P0, 1/L, epsilon)

array([[ -2.23195586e+00,   4.90901655e-01,   9.99038902e+00, ...,
         -4.74720062e-01,   3.92005611e-02,   8.67313639e-01],
       [  4.53690158e+00,  -1.35383903e+01,  -2.56918810e+00, ...,
          3.63328948e-01,  -3.15228411e-01,  -1.77354579e-01],
       [ -2.02973198e+01,  -4.69295419e-01,  -1.07439139e+01, ...,
         -3.43309414e-01,   8.20572197e-02,   1.54910736e-01],
       [  5.76413523e+01,   2.77434992e+01,   1.98610680e+01, ...,
          6.08854704e-02,   6.75456877e-01,   6.32182974e-01]])

## 3 Raffinements algorithmiques pour le problème à $Q_0$ fixé

### Question 3.1

In [7]:
def gradient_linear(g, P0, epsilon):
    """
    Minimise g par la methode du gradient avec recherche lineaire.
    Prend en entree
    g : fonction a minimiser, Pk -> g(Pk), grad_g(Pk)
    P0 : point de depart
    epsilon : critere d'arret, norme de Frobenius du gradient de g en Pk inferieur ou egal a epsilon

    Sorties :
    Pk : minimiseur
    """

    a, b, beta = 1/2, 1/2, 10**(-4) # Armijo's linear search parameters

    grad_G = g(P0)[1]
    Pk = P0
    while norm(grad_G) > epsilon:
        # Armijo's linear search
        l = 0
        while g(Pk-b*(a**l)*grad_G)[0] > g(Pk)[0] - beta*b*(a**l)*norm(grad_G):
            l = l + 1
        Pk = Pk - b*(a**l)*grad_G
        b = 2*b*(a**l)
        grad_G = g(Pk)[1]

    return Pk

In [8]:
gradient_linear(lambda P: objective(P, Q0, R, mask, rho), P0, epsilon)

array([[ -2.23867019e+00,   4.88013348e-01,   9.97161501e+00, ...,
         -4.81058941e-01,   3.97404674e-02,   8.78795758e-01],
       [  4.53543200e+00,  -1.35229851e+01,  -2.55365465e+00, ...,
          3.68183224e-01,  -3.19572407e-01,  -1.79703893e-01],
       [ -2.03074942e+01,  -4.68161609e-01,  -1.07548738e+01, ...,
         -3.47897006e-01,   8.31881918e-02,   1.56963107e-01],
       [  5.76592766e+01,   2.77584578e+01,   1.98915264e+01, ...,
          6.16997703e-02,   6.84774112e-01,   6.40565861e-01]])

### Question 3.2

On remarque que $g$ est une forme quadradique.

In [9]:
def gradient_conjugate(g, P0, epsilon):
    """
    Minimise g par la methode du gradient conjugue : Fletcher et Reeves.
    Prend en entree
    g : fonction a minimiser, Pk -> g(Pk), grad_g(Pk)
    P0 : point de depart
    
    Sorties :
    Pk : minimiseur
    """
    
    grad_G = g(P0)[1]
    d = -grad_G
    Pk = P0
    while np.linalg.norm(grad_G) > epsilon:
        s = brent(lambda s: g(Pk + s*d)[0])
        Pk = Pk + s*d
        prev_grad = grad_G
        grad_G = g(Pk)[1]
        b = np.sum(grad_G**2) / np.sum(prev_grad**2)
        d = -grad_G + b*d

        return Pk

In [10]:
gradient_conjugate(lambda p: objective(p, Q0, R, mask, rho), P0, epsilon)

array([[  2.81549597e+00,   5.65741396e-01,   8.52713169e+00, ...,
         -1.66842642e-01,   1.35359674e-02,   3.06240082e-01],
       [ -4.03630355e+00,  -1.49313803e+01,  -2.78188027e+00, ...,
          1.27563596e-01,  -1.08737554e-01,  -6.25583912e-02],
       [ -2.33915218e+01,  -1.88169409e+00,  -7.68556846e+00, ...,
         -1.20498054e-01,   2.82968797e-02,   5.46251021e-02],
       [  6.73205130e+01,   2.46845268e+01,   1.39863834e+01, ...,
          2.13376695e-02,   2.32572437e-01,   2.22583301e-01]])

### Question 3.3

Mesure des performances des algorithmes.

In [11]:
print(timeit('gradient(lambda P: objective(P, Q0, R, mask, rho), P0, 1/L, epsilon)', number=1, globals=globals()))
print(timeit('gradient_linear(lambda P: objective(P, Q0, R, mask, rho), P0, epsilon)', number=1, globals=globals()))
print(timeit('gradient_conjugate(lambda p: objective(p, Q0, R, mask, rho), P0, epsilon)', number=1, globals=globals()))

1.1569035449974763
0.842063488002168
0.2907360479985073


## 4 Résolution du problème complet

### Question 4.1

In [12]:
def total_objective(P, Q, R, mask, rho):
    """
    La fonction objectif du probleme complet.
    Prend en entree 
    P : la variable matricielle de taille C x I
    Q : la variable matricielle de taille U x C
    R : une matrice de taille U x I
    mask : une matrice 0-1 de taille U x I
    rho : un reel positif ou nul

    Sorties :
    val : la valeur de la fonction
    grad_P : le gradient par rapport a P
    grad_Q : le gradient par rapport a Q
    """

    tmp = (R - Q.dot(P)) * mask

    val = np.sum(tmp ** 2)/2. + rho/2. * (np.sum(Q ** 2) + np.sum(P ** 2))

    grad_P = rho * P - Q.T.dot(tmp)

    grad_Q = rho * Q - tmp.dot(P.T)

    return val, grad_P, grad_Q

In [13]:
 def rechercheLineaire(g,P0,Q0,epsilon):
    P=P0
    Q=Q0
    val, grad_P,grad_Q = total_objective(P=P, Q=Q, R=R , mask=mask, rho=0.3)
    norm_grad = np.sqrt(np.sum(grad_P**2)+np.sum(grad_Q**2))
    while (norm_grad > epsilon):
        l=0
        a,b,beta=(0.5,2.0,0.5)
        gamma=b
        Pprime = P - gamma*grad_P
        Qprime = Q - gamma*grad_Q
        valprime, grad_Pprime,grad_Qprime= total_objective(P=Pprime, Q=Qprime, R=R , mask=mask, rho=0.3)
        num=val+ beta*(np.trace(grad_P.T.dot(Pprime-P))+np.trace(grad_Q.T.dot(Qprime-Q)))
        while(valprime> num):
            l=l+1
            gamma=b*a**l
            Pprime = P - gamma*grad_P
            Qprime = Q - gamma*grad_Q
            valprime, grad_Pprime,grad_Qprime= g(P=Pprime, Q=Qprime, R=R , mask=mask, rho=0.3)
            num=val+ beta*(np.trace(grad_P.T.dot(Pprime-P))+np.trace(grad_Q.T.dot(Qprime-Q)))
        P=Pprime
        Q=Qprime
       
        grad_P=grad_Pprime
        grad_Q=grad_Qprime
        val=valprime
        norm_grad = np.sqrt(np.sum(grad_P**2)+np.sum(grad_Q**2))
    return val,P,Q

In [14]:
val,P,Q=rechercheLineaire(total_objective,P0,Q0,100)
print(val)

35970.7267457


La valeur retournée par l'algorithme correspond a une estimation des notes que les utilisateurs mettraient au divers films qu'ils n'ont pas encore regardés( $R = QP$)

### Question 4.2

Remarquons que la fonction est convexe en chacune des variables mais n'est pas elle-même convexe. La méthode de résolution par la méthode des moindres carrés s'appuie sur la convexité en chacune des variables. En effet on est assuré d'avoir à chaque fois une valeur objectif qui décroît car à chaque fois que l'on avance dans les itérations, la minimisation de la fonction convexe par rapport à l'une ou l'autre des deux variables permet de trouver une valeur plus petite que précédement (cela est vrai car les deux variables ne varient pas en même temps).
La fonction objectif étant minorée par 0 et décroissante , elle est donc convergente.

### Question 4.3

In [15]:
def gradientOnP(g,P0, Q0,epsilon, gamma):
    P=P0
    Q=Q0
    val, grad_P, grad_Q = total_objective(P=P, Q=Q, R=R , mask=mask, rho=0.2)
    grad = grad_P
    norm_grad = np.sqrt(np.sum(grad ** 2))
    while (norm_grad > epsilon) :
        P = P - gamma*grad
        val, grad_P, grad_Q = total_objective(P=P, Q=Q, R=R , mask=mask, rho=0.2)
        grad=grad_P
        norm_grad= np.sqrt(np.sum(grad_P ** 2))
    return val, P

def gradientOnQ(g,P0, Q0,epsilon, gamma):
    P=P0
    Q=Q0
    val, grad_P, grad_Q = total_objective(P=P, Q=Q, R=R , mask=mask, rho=0.2)
    grad = grad_Q
    norm_grad = np.sqrt(np.sum(grad ** 2))
    while (norm_grad > epsilon) :
        Q = Q - gamma*grad
        val, grad_P, grad_Q = total_objective(P=P, Q=Q, R=R , mask=mask, rho=0.2)
        grad = grad_Q
        norm_grad= np.sqrt(np.sum(grad_Q ** 2))
    return val, Q

In [16]:
Q0, s, P0 = svds(R, k=4)
Qb = Q0
Pb = P0
listeval = []
n=10000

while n > 100:
    Qstor = Qb
    val, Qb = gradientOnQ(total_objective,P0=Pb, Q0=Q0,epsilon=500, gamma=10**(-(len(listeval)+1)))
    listeval.append(val)
    Pstor = Pb
    val, Pb = gradientOnP(total_objective,P0=P0, Q0=Qb,epsilon=5000, gamma=0.000001)
    listeval.append(val)
    n = np.sqrt(np.sum((Pstor - Pb) ** 2) + np.sum((Qstor - Qb)**2)) 
    print(val)

146828.165471
80725.0327028
77331.0604341


### Question 4.4

In [17]:
R1 = Q.dot(P)
R2 = Qb.dot(Pb)
norm = np.linalg.norm((R1-R2))
norm2 = np.linalg.norm((Pb-P))
norm3 = np.linalg.norm((Qb-Q))

print("La difference de valeur de R entre la méthode du gradient et la méthode des moindres carrés est: ", norm)
print("La difference de valeur de P entre la méthode du gradient et la méthode des moindres carrés est: ", norm2)
print("La difference de valeur de Q entre la méthode du gradient et la méthode des moindres carrés est: ", norm3)

La difference de valeur de R entre la méthode du gradient et la méthode des moindres carrés est:  1703.35031205
La difference de valeur de P entre la méthode du gradient et la méthode des moindres carrés est:  74.4828602245
La difference de valeur de Q entre la méthode du gradient et la méthode des moindres carrés est:  317.886471223


### Question 4.5

In [18]:
newmask = np.logical_not(mask)
newR = R1*newmask
noteUser = newR[300,:].reshape(-1)
print("Le film recommandé pour l'utilisateur 300 est :", np.argmax(noteUser))

Le film recommandé pour l'utilisateur 300 est : 312
